In [ ]:
# General libraries
import pandas as pd
import os
import numpy as np
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

# Machine Learning Libraries
from sklearn.metrics import accuracy_score 

%matplotlib notebook

In [ ]:
loc = 'C:/Users/Utente/Desktop/Project-lab-tem-Theta-/Data/'
df = pd.read_csv(loc+ 'final_dataset.csv', index_col = [0])

In [ ]:
odd_data = pd.DataFrame(None)
odd_data = df[["MW","HomeTeam", "AwayTeam","Date"]]
odd_data

In [ ]:
# Drop useless columns for prediction 
df = df.drop(["MW","HomeTeam", "AwayTeam","Date","HM1", "HM2", "HM3", "HM4", "HM5", "AM1","AM2", "AM3", "AM4", "AM5", "HTFormPtsStr", "ATFormPtsStr"], 1)
df.head()

In [ ]:
# Keep only for the last year 
odd_data = odd_data[5700:]

odd_data

In [ ]:
## Testing set (2015-16 season)
test_data = df[5700:]
train_data = df[:5700]
# Dropping the "Result" column
X_test = test_data.drop('FTR',1)
X_train = train_data.drop('FTR',1)

In [ ]:
# Keeping only the "Result" column
y_test = test_data.FTR
y_train = train_data.FTR

In [ ]:
from xgboost import XGBClassifier
XGB = XGBClassifier(random_state = 42)
XGB.fit(X_train, y_train)

# Predict
y_pred_test = XGB.predict(X_test)

# Predict the probabilities of each class
y_pred_probability = XGB.predict_proba(X_test)

# Compute Accuracy
test_acc = accuracy_score(y_test, y_pred_test)

# Adding to the dataframe the predicted output
test_data['Prediction'] = y_pred_test
print("Test Accuracy: ", test_acc)

In [ ]:
# Creating a new dataframe with the probabilities to predict a result
df_prob = pd.DataFrame({'home_prob': [row[2] for row in y_pred_probability],
                        'draw_prob': [row[1] for row in y_pred_probability],
                        'away_prob': [row[0] for row in y_pred_probability]})

# Keeping only certains columns
test_data = test_data[['FTR','Prediction']]

# Merging together the two datasets
dataset = pd.concat([test_data.reset_index(drop = True) ,
                    odd_data.reset_index(drop = True), df_prob],1)

dataset.head(3)

In [ ]:
dataset =dataset.drop(["FTR","Date"],1)
dataset.head()

In [ ]:
print("Insert the match week for which you want a suggestion for betting:")
week= input("MW:")
week=int(week)
dataset=dataset.loc[(dataset['MW']== week)]
print(dataset)
print("lunghezza",len(dataset.MW))